In [1]:
import os
import pandas as pd
import zipfile
import pyodbc
import requests
import io
from datetime import datetime, timedelta
import sqlalchemy
import urllib

from TAPI import capturar_fecha

In [14]:
#TODO: del .mdb genero un df y lo agrego a la tabla de la BBDD

# DATOS DE LA BBDD, SERVER Y TABLA

server = 'DARCCVWSQL19'
#server = 'DESKTOP-37ESKFT\SQLEXPRESS'
database = 'TAPI'
tabla = 'DiarioTest'

# Fechas para seleccionar el día de la carga se debe iterar

Fecha_desde = "2024-03-13T00:00:00.000-03:00"
Fecha_hasta = "2024-03-13T23:59:59.000-03:00"

#URL para capturar Id del documento y el zip file:

#Defino la tabla de CAMMESA que me voy a traer
NEMO = "PARTE_POST_OPERATIVO_UNIF"
#Busco los zip disponibles para traer así puedo extraer el id
URL = f"https://api.cammesa.com/pub-svc/public/"

method_id = "findDocumentosByNemoRango?" #ID
method_zip = "findAllAttachmentZipByNemoId?"
zip_path = rf"C:\Users\Jony\OneDrive - Pampa Energia\Laburo-Estudio\Produccion\Proyectos\Tablero de Análisis de Pérdidas de Ingresos\Proyecto\.zips"
mdb_path = rf"C:\Users\Jony\OneDrive - Pampa Energia\Laburo-Estudio\Produccion\Proyectos\Tablero de Análisis de Pérdidas de Ingresos\Proyecto\.zips\.mdb"

connection_string = f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes;'

url_doc_id = f"{URL}{method_id}fechadesde={Fecha_desde}&fechahasta={Fecha_hasta}&nemo={NEMO}"

In [15]:
url_doc_id

'https://api.cammesa.com/pub-svc/public/findDocumentosByNemoRango?fechadesde=2024-03-13T00:00:00.000-03:00&fechahasta=2024-03-13T23:59:59.000-03:00&nemo=PARTE_POST_OPERATIVO_UNIF'

In [16]:
try:
    with requests.get(url_doc_id) as response:
        if response.status_code == 200:
            PPO=response.json()
            display(PPO)
            doc_id = PPO[-1]['id']
            print(doc_id)
        else:
            print("La solicitud falló con el código de estado:", response.status_code)
except requests.exceptions.RequestException as e:
    # Manejar la excepción
    print("Error al realizar la solicitud:", e)

[{'id': 'F45F9A389262D01703258AE00077C432',
  'fecha': '13/03/2024',
  'nemo': 'PARTE_POST_OPERATIVO',
  'titulo': 'Parte control post-operativo',
  'comentario': '2',
  'hora': '18:48',
  'adjuntos': [{'id': 'PO240313.zip',
    'campo': '$FILE',
    'nombre': 'PO240313-(14/03/2024 18:48:10 ZW3).zip'}],
  'version': '2024-03-14T18:48:00.000-03:00'}]

F45F9A389262D01703258AE00077C432


In [18]:
#conseguir el doc_id
try:
    with requests.get(url_doc_id) as response:
        if response.status_code == 200:
            PPO=response.json()
            doc_id = PPO[-1]['id']
            #print(doc_id)
        else:
            print("La solicitud falló con el código de estado:", response.status_code)
except requests.exceptions.RequestException as e:
    # Manejar la excepción
    print("Error al realizar la solicitud:", e)

url_zip = f"{URL}{method_zip}docId={doc_id}&nemo={NEMO}"

#descargar el .zip
try:
    with requests.get(url_doc_id) as response:
        if response.status_code == 200:
            r = requests.get(url_zip)

            # Crear un objeto ZipFile a partir del contenido descargado
            z = zipfile.ZipFile(io.BytesIO(r.content))

            # Directorio de destino para extraer los archivos ZIP
            destination_directory = ".zips"

            # Extraer todos los archivos del ZIP en el directorio específico
            z.extractall(destination_directory)
            zip_name = z.namelist()[0]

            #dia_mdb = capturar_fecha(name_zip)
            print(zip_name)
        else:
            print("La solicitud falló con el código de estado:", response.status_code)
except requests.exceptions.RequestException as e:
    # Manejar la excepción
    print("Error al realizar la solicitud:", e)

PO240313.zip


In [19]:
zip_name

'PO240313.zip'

In [20]:
# Función para capturar la fecha del nombre del archivo

def fecha_dia(nombre_archivo):
    fecha_str = nombre_archivo[2:4] + "-" + nombre_archivo[4:6] + "-" + nombre_archivo[6:8]
    #print(fecha_str)
    return datetime.strptime(fecha_str, '%y-%m-%d').date()


dia_mdb = fecha_dia(zip_name)
dia_mdb

datetime.date(2024, 3, 13)

In [21]:
#Colocar los PATHs correctos donde se traeran los archivos

zip_path = rf"C:\Users\Jadurian\OneDrive - Pampa Energia\Laburo-Estudio\Produccion\Proyectos\Tablero de Análisis de Pérdidas de Ingresos\Proyecto\.zips\{zip_name}"
mdb_path = rf"C:\Users\Jadurian\OneDrive - Pampa Energia\Laburo-Estudio\Produccion\Proyectos\Tablero de Análisis de Pérdidas de Ingresos\Proyecto\.zips\.mdb"
try:
  
    # Extrae el archivo MDB de cada archivo ZIP mensual
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Encontrar el nombre del archivo MDB dentro del ZIP mensual
        archivo_mdb = os.path.splitext(zip_name)[0] + ".mdb"
        zip_ref.extract(archivo_mdb, path=mdb_path)

    # Lee el archivo MDB y cargar la tabla VALORES_GENERADORES en un dataframe
    mdb_file = os.path.join(mdb_path, archivo_mdb)
    conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={mdb_file};"
    conn = pyodbc.connect(conn_str)
    valores_generadores = pd.read_sql("SELECT * FROM VALORES_GENERADORES", conn)
    conn.close()
    valores_generadores.insert(0, 'FECHA', dia_mdb)


except FileNotFoundError:
    print(f"El archivo {zip_name} no se encontró. Saltando al siguiente archivo...")

C:\Users\jadurian\AppData\Local\Temp\ipykernel_2692\1455759821.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  valores_generadores = pd.read_sql("SELECT * FROM VALORES_GENERADORES", conn)


In [23]:
valores_generadores.shape

(15936, 40)

In [24]:
#TODO: Cargamos el Dataframe a la tabla de sql con sqlalchemy

quoted = urllib.parse.quote_plus(connection_string)

engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

valores_generadores.to_sql('DiarioTest', schema='dbo', con=engine, if_exists='append')

24

In [14]:
valores_generadores.shape

(15864, 40)

In [15]:
#TODO: generar una tabla que sea la concatenación de todas las tablas de 'PMBD Component Degradation'"Prot_v2_TEST copy.ipynb"
contador = 0

conneccion = pyodbc.connect(connection_string)

cursor = conneccion.cursor()

for index, row in valores_generadores.iterrows():
    # Crear la consulta SQL para insertar la fila en la tabla
    query = f"INSERT INTO DiarioTest (FECHA, GRUPO, HORA, ARL, ESTADO_OPE, PI_SOTR, ARL_ECON, EST_REMUN, ENERGIA, PRECIO_NODO, PR_REM_ENERGIA, SMEC, POT_DISP, POT_OPE_CPO, COMPRA_SPOT, SCTD, SCO, PIND, PINDPROG, PINDFORZ, POT_DISP_GAS, POT_DISP_RESERVA, GAS_NOMINADO, COSTO_406, REM_ADICIONAL, DESP_ECON, CCM, CFO, CGN, CGO, POT_DISP_R19, FACTOR_KM, HORA_REPRESENTATIVA, ENERG_GEN_TRANSAC, ENERG_OPE_TRANSAC, SOBREC_FORZ_PROPIO, PGENE_RES354, CGU_PROPIO, CGU_MEM, CGU_GAS_CEDIDO) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    
    # Ejecutar la consulta SQL con los valores de la fila actual
    cursor.execute(query, tuple(row))
    # Confirmar los cambios
    conneccion.commit()

# Cerrar el cursor y la conexión
cursor.close()
conneccion.close()

ProgrammingError: ('The SQL contains 39 parameter markers, but 40 parameters were supplied', 'HY000')